## Hotlap Analysis - 2022 vs. 2021

In 2022 F1 introduced new regulations completely overhauling the design of F1 cars. Over a single lap, what has changed? I'll analyze a lap from a Ferrari driver to see what we can determine.

In [1]:
#Import packages
import pandas as pd
import numpy as np
import fastf1 as ff1
#Enable the cache
ff1.Cache.enable_cache('C:/Users/jackh/OneDrive/Documents/Python Scripts/f1_cache') 

In [2]:
#Get the data
#Get data
quali_2021 = ff1.get_session(2021, 'Bahrain', 'Q')
quali_2022 = ff1.get_session(2022, 'Bahrain', 'Q')

#Get the laps
laps_21 = quali_2021.load_laps(with_telemetry=True)
laps_22 = quali_2022.load_laps(with_telemetry=True)

#Select all laps from Charles Leclerc in both 2021 & 2022
lec_2021 = laps_21.pick_driver('LEC')
lec_2022 = laps_22.pick_driver('LEC')

#Get their fastest data telemetry
fastest_2021 = lec_2021.pick_fastest().get_telemetry().add_distance()
fastest_2022 = lec_2022.pick_fastest().get_telemetry().add_distance()

#And add the driver variable
fastest_2021['Year'] = '2021'
fastest_2022['Year'] = '2022'


#Finally, merge both objects into a single dataframe
telemetry = fastest_2021.append(fastest_2022)

telemetry.head()



core           INFO 	Loading laps for Bahrain Grand Prix - Qualifying [v2.1.13]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 20 drivers: ['47', '14', '10', '99', '44', '7', '63', '9', '11', '55', '31', '18', '5', '22', '4', '33', '77', '16', '3', '6']
core           INFO 	Loading laps for Bahrain Grand Prix - Qualifying [v2.1.13]
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api    

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,RelativeDistance,Status,X,Y,Z,Distance,Year
2,2021-03-27 15:59:12.501,0 days 01:14:12.116000,,432.520833,0 days 00:00:00,10584,295,8,100,0,12,interpolation,0.000006,OnTrack,-382,1200,-155,0.000000,2021
3,2021-03-27 15:59:12.644,0 days 01:14:12.259000,,432.520833,0 days 00:00:00.143000,10604,296,8,100,0,12,pos,0.002186,OnTrack,-378,1318,-159,11.757778,2021
4,2021-03-27 15:59:12.675,0 days 01:14:12.290000,,432.520833,0 days 00:00:00.174000,10645,297,8,100,0,12,car,0.002660,OnTrack,-376,1343,-159,14.315278,2021
5,2021-03-27 15:59:12.864,0 days 01:14:12.479000,,432.520833,0 days 00:00:00.363000,10676,297,8,100,0,12,pos,0.005549,OnTrack,-369,1499,-159,29.907778,2021
6,2021-03-27 15:59:12.915,0 days 01:14:12.530000,33,432.520833,0 days 00:00:00.414000,10708,298,8,100,0,12,car,0.006331,OnTrack,-367,1541,-158,34.129444,2021


#### Creating Minisectors

Since we don't have telemetry data at exactly the same spots from session to session, one helpful way to standardize the results is to break the hot laps into 25 minisectors. I will do this below.

In [3]:
#Define number of minisectors
num_minisectors = 25

#Get max distance on the track
total_distance = total_distance = max(telemetry['Distance'])

#Generate equally-sized mini-sectors
minisector_length = total_distance/num_minisectors

#Now we can find the point at which each minisector starts
minisectors = [0]

for i in range(0, (num_minisectors-1)):
    minisectors.append(minisector_length * (i+1))

#And we can figure out where in each minisector the car was at the moment the data point was recorded
telemetry['Minisector'] = telemetry['Distance'].apply(
    lambda dist: (
        int((dist // minisector_length) + 1)
    )
)

telemetry.head()